In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from collections import defaultdict
from tqdm import tqdm
from sklearn.decomposition import PCA
import sys
import xgboost as xgb
#Dataset=pd.DataFrame([{'id':1 , '2':100}]) #исходный pandas-dataframe , содержащий айди картинки и вектора к ним и
                        #результат свайпа пользователя 

batch_size = 10

#DictData={'id1':[1 , 2 , 3 , ..] , 'id2': [2 , 4 ,6 , ..]}
#TargetData = {'id1' : '1' , 'id2': '2'}
#DeliveredData == TargetData

def CalculateScore(clf):
    clf

In [71]:
print(sys.getsizeof(imagemap))
print(sys.getsizeof(dataset))

1310816
108404


In [72]:
class ServerUser:
    def __init__(self):
        self.UsedGirls = set()
        self.innerVector = [] #Если будем все-таки делать через статистики
        embvecs = pd.read_csv('insight_face_model/vecs2.csv')
        dataset = pd.read_csv('likes.csv')
        imagemap = {}
        for index, row in tqdm(embvecs.iterrows()):
            vector = row.values[0:512]
            name = row.values[512]
            imagemap[name] = vector
        self.clf = SVC(kernel='linear' , C = 100 , random_state = 241)#Потестить  svm кросс-валидацие
        
    def getLeftData(self, data):
        returnableDataset = pd.DataFrame(Dataset)
        for index, row in data.iterrows():
            returnableDataset = returnableDataset[returnableDataset['image_name'] != row['image_name'] and 
                                                 returnableDataset['image_name'] not in UsedGirl]
            self.UsedGirls[row["image_name"]] = True
        return returnableDataset 
         #вернуть новую таблицу, состоящую из разницы Dataset/data
        
    def getData(self, likes):
        for index, row in likes.iterrows():
            if(row['image_name'] in DictData):
                self.data = self.data.append(pd.DataFrame([{'vectors' : DictData[row["image_name"]] , 'target' : row['target']}], columns=['vectors', 'target']))

        self.clf.fit(self.data['vectors'].values , self.data['target'].values)
        LeftData = getLeftData(self.data)
        predictedData = self.clf.predict_log_proba(LeftData)
        LeftData['prediction'] = pd.Series(predictedData , index = LeftData.index)
        LeftData.sort(column='prediction' ,  ascending=False) # Отсортировать лист по <x, w> * y
        UserArray = LeftData.head(8)#Выбрать первые 3/4 представителей от batch_size
        UserArray.append(LeftData.sample(2)) #рандомим остальные
        
        return UserArray['image_name'].to_list() #Отослать юзеру эти 5 представителей (или сколько там батч у нас)
        
    def sendToFrontendRandomSamples(self):
        randomSamples = Dataset.sample(batch_size)
        return(randomSamples["image_name"].tolist())
        
    def getScoreFromRandomSamples(self , json): #Принимаем таблицу с рандом семплами и таргетами
        randomSamplesWithScore = JSONToDataFrame(json)
        Rand = randomSamplesWithScore.shape['0']      

In [100]:
User = ServerUser()
User.getData(TargetData)

ValueError: setting an array element with a sequence.

In [44]:
embvecs = pd.read_csv('insight_face_model/vecs2.csv')
dataset = pd.read_csv('likes.csv')
df_res = embvecs['image_name'].copy()
usedGirls = set()
imagemap = {}
for index, row in tqdm(embvecs.iterrows()):
    vector = row.values[0:512]
    name = row.values[512]
    usedGirls.add(name)
    imagemap[name] = vector
clf = SVC(C = 0.01 , random_state = 241,probability=True)#Потестить  svm кросс-валидацие

37125it [00:01, 23020.56it/s]


In [45]:
X_train = []
y_train = []
for i in tqdm(range(dataset.shape[0])):
    if dataset['image_name'].iloc[i] in imagemap:
        X_train.append(imagemap[dataset['image_name'].iloc[i]])
        y_train.append(dataset['target'].iloc[i])
X_train = np.array(X_train)
y_train = np.array(y_train)

100%|███████████████████████████████████████████████████████████████████████████| 1071/1071 [00:00<00:00, 39772.45it/s]


In [46]:
%%time
clf.fit(X_train, y_train)

C:\Users\Nik\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Wall time: 706 ms


SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

In [47]:
%%time
res = (clf.predict_proba(X_train[300:352]))[:,1]

Wall time: 9.97 ms


In [48]:
c = 0
for i in res:
    if i>0.25:
        c+=1
c

0

In [49]:
np.sort(res)

array([0.20996434, 0.20996737, 0.21002165, 0.21026583, 0.21057826,
       0.2107054 , 0.21085462, 0.21117029, 0.21135941, 0.21141869,
       0.21143453, 0.2114513 , 0.21149361, 0.21163696, 0.21172839,
       0.21192453, 0.21199903, 0.21225088, 0.21244111, 0.21250165,
       0.2125618 , 0.21268605, 0.21285464, 0.21308001, 0.21320539,
       0.21341489, 0.21342811, 0.21351244, 0.21351701, 0.2138532 ,
       0.21388789, 0.21401668, 0.21408641, 0.21415583, 0.21415826,
       0.21444149, 0.21448432, 0.21484524, 0.21499799, 0.21504073,
       0.21504118, 0.21516723, 0.21521363, 0.21574499, 0.21589217,
       0.21591492, 0.21631343, 0.21633154, 0.21651049, 0.21721312,
       0.21799971, 0.22021939])

In [51]:
%%time
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

Wall time: 2.53 s


In [52]:
%%time
res = model.predict_proba(X_train[300:352])[:,1]
res

Wall time: 19.9 ms


In [147]:
df_res = pd.DataFrame({'image_name':embvecs['image_name'].values,'proba':res})
df_res = df_res.sort_values(by=['proba'], ascending=False)
df_res

,image_name,proba
18529,10261121_126954760838400_1653301441_a,0.881934
27022,1527651_755688407783369_292834404_a,0.854834
7937,10175235_1438156093093414_1204736923_a,0.844741
13763,10249163_228889637307848_1867003485_a,0.844490
1675,10009256_562167387214362_1621936687_a,0.842641
22541,1209474_218148061728044_498217191_a,0.827364
17061,10254213_230981817092513_1814216556_a,0.823538
24737,1515869_685718098140981_2078033324_a,0.812284
23241,1389482_391635880979578_823895643_a,0.811956
18944,10261220_1439568376286267_1716150414_a,0.810464
